In [1]:
import pickle
import os
import time
import numpy as np
import os 
import shutil

import pyximport; pyximport.install()
from LEF_Dynamics import LEFTranslocatorDirectional

import warnings
import h5py 
import glob

/home/magus/anaconda3/lib/python3.7/site-packages/Cython/Compiler/Main.py:369: FutureWarning: Cython directive 'language_level' not set, using 2 for now (Py2). This will change in a later release! File: /home/magus/libs_tmp/polychrom/examples/loopExtrusion/LEF_Dynamics.pyx
  tree = Parsing.p_module(s, pxd, full_module_name)


## Defining simulation parameters

In [6]:

# -------defining parameters----------
#  -- basic loop extrusion parameters
SEPARATION = 400
LIFETIME = 200
N1 = 4000   # number of monomers
M = 10 
N = N1 * M 
smcStepsPerBlock = 1  # now doing 1 SMC step per block 

trajectoryLength=100000


birthArray = np.zeros(N1, dtype=np.double) + 0.1
deathArray = np.zeros(N1, dtype=np.double) + 1. / LIFETIME
stallDeathArray = np.zeros(N1, dtype=np.double) + 1 / LIFETIME
pauseArray = np.zeros(N1, dtype=np.double)

stallList = [500, 1500, 2500, 3500]
stallLeftArray = np.zeros(N1, dtype = np.double)
stallRightARray = np.zeros(N1, dtype = np.double)
for i in stallList:
    stallLeftArray[i] = 0.8
    stallRightARray[i] = 0.8

LEFNum = N // SEPARATION
LEFTran = LEFTranslocatorDirectional(
    np.tile(birthArray,M), 
    np.tile(deathArray,M),
    np.tile(stallLeftArray,M),
    np.tile(stallRightARray,M),
    np.tile(pauseArray,M),
    np.tile(stallDeathArray,M),
    LEFNum)
folder = "trajectory"
if not os.path.exists(folder):
    os.mkdir(folder)
    
with h5py.File("trajectory/LEFPositions.h5", mode='w') as myfile:

    dset = myfile.create_dataset("positions", 
                                 shape=(trajectoryLength, LEFNum, 2), 
                                 dtype=np.int32, 
                                 compression="gzip")
    steps = 50 
    bins = np.linspace(0, trajectoryLength, steps, dtype=int)
    for st,end in zip(bins[:-1], bins[1:]):
        cur = []
        for i in range(st, end):
            LEFTran.steps(1)
            LEFs = LEFTran.getLEFs()
            cur.append(np.array(LEFs).T)
        cur = np.array(cur)
        dset[st:end] = cur
    myfile.attrs["N"] = N
    myfile.attrs["LEFNum"] = LEFNum




In [7]:
with h5py.File("trajectory/LEFPositions.h5", mode='r') as myfile:

    dset = myfile["positions"][:]
dset

array([[[37653, 37655],
        [31866, 31868],
        [22973, 22976],
        ...,
        [ 3177,  3180],
        [31680, 31682],
        [33374, 33376]],

       [[37652, 37656],
        [31865, 31869],
        [22972, 22977],
        ...,
        [ 3176,  3181],
        [31679, 31683],
        [33373, 33377]],

       [[37651, 37657],
        [31864, 31870],
        [22971, 22978],
        ...,
        [ 3175,  3182],
        [31678, 31684],
        [33372, 33378]],

       ...,

       [[37047, 37499],
        [21501, 21814],
        [ 9097,  9117],
        ...,
        [ 6747,  7053],
        [ 4500,  4835],
        [ 5016,  5233]],

       [[37046, 37499],
        [21500, 21815],
        [ 9096,  9118],
        ...,
        [ 6746,  7054],
        [ 4500,  4836],
        [ 5016,  5233]],

       [[37045, 37499],
        [21500, 21816],
        [ 9095,  9119],
        ...,
        [ 6745,  7055],
        [ 4500,  4837],
        [ 5016,  5233]]], dtype=int32)

In [8]:
dset.shape

(100000, 100, 2)